# 任务 2a：使用  Amazon Titan Text Premier 为小型文件生成文本摘要

在此笔记本中，您将一小段文本字符串直接摄取到 Amazon Bedrock API 中（使用 Titan Text 模型），并指示它为输入文本生成摘要。当输入文本长度在模型的上下文大小限制范围之内时，您可以应用这种方法为通话记录、会议记录、书籍、文章、博客文章和其他相关内容生成摘要。

## 任务 2a.1：环境设置

在此任务中，您将设置环境。

In [ ]:
#Create a service client by name using the default session.
import json
import os
import sys
import warnings

import boto3
import botocore

warnings.filterwarnings('ignore')
module_path = ".."
sys.path.append(os.path.abspath(module_path))

bedrock_client = boto3.client('bedrock-runtime',region_name=os.environ.get("AWS_DEFAULT_REGION", None))



## 任务 2a.2：为要生成摘要的文本编写提示词

在此任务中，您将使用一小段文本，其 Token 数小于基础模型支持的最大长度。您将使用宣布 Amazon Bedrock 的 [AWS Blog 文章](https://aws.amazon.com/jp/blogs/machine-learning/announcing-new-tools-for-building-with-generative-ai-on-aws/) 中的一段话，作为本实验的输入文本示例。

提示词的开头是一项指令：`Please provide a summary of the following text.`。

In [ ]:
prompt_data = """

Please provide a summary of the following text:

AWS took all of that feedback from customers, and today we are excited to announce Amazon Bedrock, \
a new service that makes FMs from AI21 Labs, Anthropic, Stability AI, and Amazon accessible via an API. \
Bedrock is the easiest way for customers to build and scale generative AI-based applications using FMs, \
democratizing access for all builders. Bedrock will offer the ability to access a range of powerful FMs \
for text and images—including Amazons Titan FMs, which consist of two new LLMs we’re also announcing \
today—through a scalable, reliable, and secure AWS managed service. With Bedrock’s serverless experience, \
customers can easily find the right model for what they’re trying to get done, get started quickly, privately \
customize FMs with their own data, and easily integrate and deploy them into their applications using the AWS \
tools and capabilities they are familiar with, without having to manage any infrastructure (including integrations \
with Amazon SageMaker ML features like Experiments to test different models and Pipelines to manage their FMs at scale).
"""

## 任务 2a.3：使用提示词和推理参数创建请求体 

在此任务中，您将使用上述提示词和推理参数创建请求体。

In [ ]:
# request body
body = json.dumps({
    "inputText": prompt_data, 
    "textGenerationConfig":{
        "maxTokenCount":2048,
        "stopSequences":[],
        "temperature":0,
        "topP":0.9
        }
    }) 

## 任务 2a.4：通过 Boto3 调用基础模型

在此任务中，您将向 Amazon Bedrock 发送一个 API 请求，并指定请求参数 `modelId`、`accept` 和 `contentType`。随后，Amazon Bedrock 中的基础模型会按照提供的提示词为输入文本生成摘要。

### 完整输出生成

默认情况下，Amazon Bedrock 服务会在单个输出中为给定提示词生成完整摘要。如果模型输出包含许多 Token，则生成速度可能会很慢。

In [ ]:
#model configuration and invoke the model
modelId = 'amazon.titan-text-premier-v1:0' # change this to use a different version from the model provider
accept = 'application/json'
contentType = 'application/json'
outputText = "\n"

try:

    response = bedrock_client.invoke_model(body=body, modelId=modelId, accept=accept, contentType=contentType)
    response_body = json.loads(response.get('body').read())

    outputText = response_body.get('results')[0].get('outputText')

except botocore.exceptions.ClientError as error:
    
    if error.response['Error']['Code'] == 'AccessDeniedException':
           print(f"\x1b[41m{error.response['Error']['Message']}\
                \nTo troubeshoot this issue please refer to the following resources.\
                 \nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\
                 \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n")
        
    else:
        raise error

print(outputText)

### 流式传输输出生成

接下来，您将探索如何使用 Amazon Bedrock 的 invoke_model_with_response_stream API 来流式传输模型输出，以便用户在输出生成时就可以使用输出。此 API 并不会一次性生成完整输出，而是返回一个 ResponseStream，它会不断传输模型生成的较小输出块。您可以在连续的可使用视图中看到这些流式传输输出。

In [ ]:
#invoke model with response stream
modelId = 'amazon.titan-text-premier-v1:0'
response = bedrock_client.invoke_model_with_response_stream(body=body, modelId=modelId, accept=accept, contentType=contentType)
stream = response.get('body')
output = list(stream)
output

In [ ]:
from IPython.display import display_markdown,Markdown,clear_output

In [ ]:
modelId = 'amazon.titan-text-premier-v1:0'
response = bedrock_client.invoke_model_with_response_stream(body=body, modelId=modelId, accept=accept, contentType=contentType)
stream = response.get('body')
output = []
i = 1
if stream:
    for event in stream:
        chunk = event.get('chunk')
        if chunk:
            chunk_obj = json.loads(chunk.get('bytes').decode())
            text = chunk_obj['outputText']
            clear_output(wait=True)
            output.append(text)
            display_markdown(Markdown(''.join(output)))
            i+=1

现在，您已尝试使用了 boto3 SDK 来访问 Amazon Bedrock API。此 SDK 提供对 Bedrock 功能的基本编程访问。通过利用此 API，您可以实现两个使用案例：1) 一次性生成 AWS 新闻内容的完整文本摘要，以及 2) 以块形式流式传输摘要输出以进行增量处理。

### 自行尝试
- 将提示词更改为您的特定使用案例，并评估不同模型的输出。
- 尝试不同的 Token 长度，了解服务的延迟和响应能力。
- 应用不同的提示词工程原则，获得更好的输出。

### 清理

您已完成此笔记本。要进入本实验的下一部分，请执行以下操作：

- 关闭此笔记本文件并继续执行 **Task2b_zh_cn.ipynb**。